In [15]:
import kagglehub
import shutil
import wave
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy.signal import resample, spectrogram
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras.utils import to_categorical

In [ ]:
# Download latest version
path = kagglehub.dataset_download("shreyj1729/best-of-watkins-marine-mammal-sound-database")

print("Path to dataset files:", path)

In [ ]:
process_audio_data(data_path, output_folder=modified_data_folder, target_sr=target_sr)

In [ ]:
#Used to get data into same directory
#shutil.move(path, "/Users/miku/Documents/GitHub/Marine_Mammal_Classification_By_Sound")
#path = 

path = "/Users/miku/.cache/kagglehub/datasets/shreyj1729/best-of-watkins-marine-mammal-sound-database/versions/3/data"

In [ ]:
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    
    #Skip if it's not a directory
    if not os.path.isdir(folder_path):
        continue
    
    #Count the number of files in the current directory
    num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    
    print(f"Folder: {folder}, Number of files: {num_files}")


In [ ]:
def move_problematic_file(file_path, problematic_folder):
    try:
        os.makedirs(problematic_folder, exist_ok=True)
        shutil.move(file_path, os.path.join(problematic_folder, os.path.basename(file_path)))
        print(f"Moved {file_path} to {problematic_folder}")
    except Exception as e:
        print(f"Error moving file {file_path}: {e}")

In [ ]:
def process_audio_file(file_path, target_length_seconds=5, output_folder=modified_data_folder, problematic_folder=problematic_folder):
    try:
        # Read the WAV file
        sample_rate, data = wavfile.read(file_path)
        
        # Calculate the target number of samples (5 seconds)
        target_samples = target_length_seconds * sample_rate
        
        # Trim or pad the audio data
        if len(data) > target_samples:
            # Trim to 5 seconds
            data = data[:target_samples]
        elif len(data) < target_samples:
            # Pad with zeros (silence) if less than 5 seconds
            padding = target_samples - len(data)
            data = np.pad(data, (0, padding), mode='constant')

        # Save the modified file to the output folder (don't overwrite the original)
        output_file = os.path.join(output_folder, os.path.basename(file_path))
        wavfile.write(output_file, sample_rate, data)
        print(f"Processed {file_path}, saved to {output_file}")

    except ValueError as e:
        print(f"Error processing {file_path}: {e}")
        move_problematic_file(file_path, problematic_folder)

In [ ]:
def print_audio_length(filename):
    try:
        sample_rate, data = wavfile.read(filename)
        audio_length = len(data) / sample_rate
        return audio_length
    except ValueError as e:
        print(f"Error processing {filename}: {e}")
        return None

In [ ]:
def get_wav_channels(filename):
    with wave.open(filename, 'r') as wav_file:
        return wav_file.getnchannels()

In [ ]:
def resample_audio(audio_data, original_sr, target_sr=16000):
    num_samples = int(len(audio_data) * float(target_sr) / original_sr)
    return resample(audio_data, num_samples)

In [ ]:
def save_spectrogram(filename, output_base_folder, target_sr=16000):
    try:
        sample_rate, data = wavfile.read(filename)
        
        if sample_rate != target_sr:
            data = resample_audio(data, sample_rate, target_sr)
            sample_rate = target_sr
        
        f, t, Sxx = spectrogram(data, sample_rate)
        
        # Get the relative path of the animal folder from the original file
        animal_folder = os.path.basename(os.path.dirname(filename))
        
        # Define the output folder for this animal, preserving the directory structure
        animal_output_folder = os.path.join(output_base_folder, animal_folder)
        
        os.makedirs(animal_output_folder, exist_ok=True)
        
        # Define the output path for the spectrogram data file
        output_filename = os.path.join(animal_output_folder, os.path.basename(filename).replace('.wav', '_spectrogram.npy'))
        
        # Save the spectrogram as a NumPy array
        np.save(output_filename, Sxx)
        print(f"Saved spectrogram data for {filename} to {output_filename}")
    except ValueError as e:
        print(f"Error processing {filename}: {e}")

In [ ]:
def process_audio_data(data_path, output_folder=modified_data_folder, target_sr=16000):
    one_channel_count = 0
    multi_channel_count = 0

    for folder in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder)
        
        if not os.path.isdir(folder_path):
            continue
        
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            if not file.endswith('.wav'):
                print("Not wav:", file)
                continue

            sample_rate, data = wavfile.read(file_path)

            # Calculate the length of the audio in seconds
            audio_length = len(data) / sample_rate
            
            # Skip files longer than 5 seconds
            if audio_length > 5:
                print(f"Skipping file {file} as it is longer than 5 seconds.")
                continue

            num_samples_5_sec = int(sample_rate * 5)
            data_first_5_sec = data[:num_samples_5_sec]

            num_channels = get_wav_channels(file_path)
            
            # Process single-channel and multi-channel files
            if num_channels == 1:
                one_channel_count += 1
                save_spectrogram(file_path, spectrogram_folder, target_sr)
                process_audio_file(file_path, target_length_seconds=5, output_folder=modified_data_folder)
            else:
                multi_channel_count += 1
                save_spectrogram(file_path, spectrogram_folder, target_sr)
                process_audio_file(file_path, target_length_seconds=5, output_folder=modified_data_folder)
                
    print(f"Number of files with 1 channel: {one_channel_count}")
    print(f"Number of files with more than 1 channel: {multi_channel_count}")

In [ ]:
spectrogram_folder = "/Users/miku/.cache/kagglehub/datasets/shreyj1729/best-of-watkins-marine-mammal-sound-database/versions/3/spectrograms"
modified_data_folder = "/Users/miku/.cache/kagglehub/datasets/shreyj1729/best-of-watkins-marine-mammal-sound-database/versions/3/modifieddata"
problematic_folder = "/Users/miku/.cache/kagglehub/datasets/shreyj1729/best-of-watkins-marine-mammal-sound-database/versions/3/badfiles"
data_path = "/Users/miku/.cache/kagglehub/datasets/shreyj1729/best-of-watkins-marine-mammal-sound-database/versions/3/data"
target_sr = 16000

In [ ]:
os.makedirs(modified_data_folder, exist_ok=True)
os.makedirs(spectrogram_folder, exist_ok=True)
os.makedirs(problematic_folder, exist_ok=True)

In [ ]:
process_audio_data(data_path, output_folder=modified_data_folder, target_sr=target_sr)

In [ ]:
import numpy as np
import os
from scipy.ndimage import zoom

def load_spectrogram_data(spectrogram_folder, target_shape=(128, 128)):
    spectrograms = []
    labels = []
    
    for animal_folder in os.listdir(spectrogram_folder):
        animal_folder_path = os.path.join(spectrogram_folder, animal_folder)
        
        if not os.path.isdir(animal_folder_path):
            continue
        
        for file in os.listdir(animal_folder_path):
            if file.endswith('.npy'):
                file_path = os.path.join(animal_folder_path, file)
                spectrogram = np.load(file_path)
                
                # Resize spectrogram to the target shape
                spectrogram_resized = zoom(spectrogram, (target_shape[0] / spectrogram.shape[0], target_shape[1] / spectrogram.shape[1]), order=3)
                
                # Ensure spectrogram has the right dimensions (e.g., 2D: time x frequency)
                spectrograms.append(spectrogram_resized)
                labels.append(animal_folder)
    
    # Convert lists to numpy arrays
    spectrograms = np.array(spectrograms)
    labels = np.array(labels)
    
    return spectrograms, labels

# Load the spectrogram data with the resized shape
spectrograms, labels = load_spectrogram_data(spectrogram_folder, target_shape=(128, 128))

# Check the shapes
print(f"Spectrograms shape: {spectrograms.shape}")
print(f"Labels shape: {labels.shape}")
